In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tweets-lemmatized/lemmatized_tweets.csv
/kaggle/input/w2v100/Word2vec_100 data.csv


In [2]:
import pandas as pd
import numpy as np

In [3]:
from tqdm import tqdm

In [4]:
data = pd.read_csv("/kaggle/input/tweets-lemmatized/lemmatized_tweets.csv")

In [5]:
data.head()

,Unnamed: 0.1,Unnamed: 0,index,Date,text,Sentiment,counts
0,0,0,0,2019-05-27,appena uscito un nuovo video le cryptomonnaies...,Positive,1
1,1,1,1,2019-05-27,cardano digitize currency eos roi amp bitcoin ...,Positive,2
2,2,2,2,2019-05-27,another test tweet caught stream bitcoin,Positive,3
3,3,3,3,2019-05-27,current crypto price btc usd eth usd ltc usd b...,Positive,4
4,4,4,4,2019-05-27,spiv nosar baz bitcoin asset amp currency,Positive,5


In [ ]:
import gensim
from gensim.models import Word2Vec, KeyedVectors

In [ ]:
arr = []
for i in range(len(data)):
    arr.append(data['text'][i])

In [ ]:
arr[0]

In [ ]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [ ]:
words = []
for sent in arr:
    sent_token = sent_tokenize(str(sent))
    for sent in sent_token:
        words.append(simple_preprocess(sent))

In [ ]:
words[0]

In [ ]:
# training gensim model
model = gensim.models.Word2Vec(words, vector_size = 100, epochs = 100)

In [ ]:
model.corpus_count

In [ ]:
def avg_word2vec(doc):
    return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key ],axis=0)

In [ ]:
# applying word2vec on all sentences
x = []
for i in tqdm(range(len(words))):
    x.append(avg_word2vec(words[i]))

In [ ]:
len(x)

In [ ]:
x_new = np.array(x)

In [ ]:
x_new[0]

In [ ]:
df=pd.DataFrame()
for i in tqdm(range(0,len(x))):
    df=df.append(pd.DataFrame(x_new[i].reshape(1,-1)),ignore_index=True)

In [ ]:
df.to_csv('Word2vec_100 data.csv')

In [6]:
word2vec = pd.read_csv("/kaggle/input/w2v100/Word2vec_100 data.csv")

In [7]:
word2vec.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,0,0.770586,0.778003,-0.511993,0.908083,-0.145112,-1.269499,-0.110509,-0.206192,-0.671741,...,-0.327191,-0.342318,-0.673540,2.140765,-0.094885,-1.009921,0.179221,-0.319410,1.043736,-0.147896
1,1,1.523856,-1.282496,-1.314860,-0.855154,-1.614949,0.905590,-0.628848,-0.370854,1.825447,...,0.018768,-0.251574,-0.358567,0.013824,0.301618,-0.827346,0.575639,-0.371606,-1.251194,-0.511509
2,2,-0.807502,0.156672,-0.477368,0.827913,-1.390720,-0.986676,0.704052,-0.290838,1.158290,...,1.227507,-0.568289,-0.495195,1.060969,-1.987696,-0.480395,1.152764,-0.605510,-0.074227,-0.220436
3,3,0.727483,0.099754,0.173596,-2.292075,-1.237633,3.805687,0.545229,0.886751,0.731681,...,1.175219,0.532808,0.419393,-0.498257,-2.097116,1.084193,1.652125,1.667037,3.358975,0.030423
4,4,1.117927,-0.691438,-2.187568,0.835525,-0.858302,0.180846,1.136570,0.124332,1.990469,...,0.617761,-0.173047,1.046192,-0.287051,-0.882932,-1.814162,0.782317,-0.927441,-0.781292,0.689507


In [8]:
word2vec.isna().sum().sum()

276300

In [9]:
y = data['Sentiment']

In [10]:
word2vec["sentiment"] = y

In [11]:
word2vec = word2vec.dropna().reset_index(drop=True)

In [12]:
word2vec.isna().sum().sum()

0

In [13]:
word2vec.shape

(465973, 102)

In [14]:
y = word2vec['sentiment']

In [15]:
y = pd.get_dummies(y)

In [16]:
y

,Negative,Neutral,Positive
0,0,0,1
1,0,0,1
2,0,0,1
3,0,0,1
4,0,0,1
...,...,...,...
465968,0,0,1
465969,0,0,1
465970,0,0,1
465971,0,0,1


In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional
from sklearn.decomposition import PCA

In [18]:
x = word2vec.iloc[:, 1:101]

In [19]:
x

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.770586,0.778003,-0.511993,0.908083,-0.145112,-1.269499,-0.110509,-0.206192,-0.671741,0.850975,...,-0.327191,-0.342318,-0.673540,2.140765,-0.094885,-1.009921,0.179221,-0.319410,1.043736,-0.147896
1,1.523856,-1.282496,-1.314860,-0.855154,-1.614949,0.905590,-0.628848,-0.370854,1.825447,-0.700947,...,0.018768,-0.251574,-0.358567,0.013824,0.301618,-0.827346,0.575639,-0.371606,-1.251194,-0.511509
2,-0.807502,0.156672,-0.477368,0.827913,-1.390720,-0.986676,0.704052,-0.290838,1.158290,-0.454432,...,1.227507,-0.568289,-0.495195,1.060969,-1.987696,-0.480395,1.152764,-0.605510,-0.074227,-0.220436
3,0.727483,0.099754,0.173596,-2.292075,-1.237633,3.805687,0.545229,0.886751,0.731681,1.196639,...,1.175219,0.532808,0.419393,-0.498257,-2.097116,1.084193,1.652125,1.667037,3.358975,0.030423
4,1.117927,-0.691438,-2.187568,0.835525,-0.858302,0.180846,1.136570,0.124332,1.990469,-1.844361,...,0.617761,-0.173047,1.046192,-0.287051,-0.882932,-1.814162,0.782317,-0.927441,-0.781292,0.689507
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465968,-0.538646,2.003169,0.603866,-3.139169,-0.357094,2.405778,-1.336510,2.684112,1.242054,0.384833,...,1.155718,0.367652,1.575840,-0.658323,0.225536,-2.604613,3.277756,-0.297232,1.665336,0.628544
465969,0.258216,-0.292319,-0.228997,-0.631366,-0.520732,0.602971,0.393577,0.410106,0.258083,0.729874,...,-0.520287,-0.502319,-1.960005,1.761033,-1.325427,0.308235,0.780936,-0.730051,-0.595190,0.553579
465970,-1.450608,-0.136676,0.674319,-0.564774,-1.856690,1.399925,-0.157177,0.183732,1.729790,-1.092956,...,0.062751,1.284911,-0.170330,0.275123,-0.810206,-0.220326,0.643079,-0.817995,0.858107,0.242071
465971,-1.440638,-0.678630,0.964140,0.207578,-0.618750,0.740476,-0.309027,0.764684,0.580091,-1.356656,...,0.409391,-0.112722,0.585394,-0.737284,-1.039693,-1.871212,1.687462,-0.912822,-0.054739,0.131407


In [20]:
pca = PCA(n_components = 15)

In [21]:
pca.fit(x)

PCA(n_components=15)

In [22]:
x_pca = pca.transform(x)

In [23]:
x_pca.shape

(465973, 15)

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
x_train, x_test, y_train, y_test = train_test_split(x_pca, y,test_size=0.2, shuffle=False)

In [26]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((372778, 15), (372778, 3), (93195, 15), (93195, 3))

In [ ]:
# x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
# x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

In [ ]:
# x_train.shape, x_test.shape

In [ ]:
# model = Sequential()
# # model.add(Bidirectional(LSTM(256,recurrent_dropout=0.1),input_shape = (2,1)))
# model.add(LSTM(256, return_sequences = True, input_shape = (100,1)))
# model.add(LSTM(128, return_sequences = True))
# model.add(LSTM(64))
# # model.add(LSTM(32))
# model.add(Dense(3, activation = 'softmax'))
# model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

In [ ]:
# model.summary()

In [ ]:
# model.fit(x_train,y_train, validation_data=(x_test,y_test), epochs=200, batch_size=50, verbose=1)

In [ ]:
# y_pred = model.predict(x_test)

In [ ]:
# from sklearn.metrics import accuracy_score

In [ ]:
# score = model.evaluate(x_test, y_test)

In [ ]:
# score

In [26]:
# from sklearn.ensemble import RandomForestClassifier

In [42]:
# classifier = RandomForestClassifier(n_estimators = 500)

In [43]:
# classifier.fit(x_train, y_train)

RandomForestClassifier(n_estimators=500)

In [44]:
# y_pred = classifier.predict(x_test)

In [45]:
# from sklearn import metrics 
# print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y_test, y_pred))

ACCURACY OF THE MODEL:  0.5196952626213852


In [28]:
# from xgboost import XGBClassifier

In [29]:
# model = XGBClassifier()
# model.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [31]:
# y_pred = model.predict(x_test)

In [32]:
# from sklearn import metrics 
# print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y_test, y_pred))

ACCURACY OF THE MODEL:  0.5211760287569076


In [34]:
# from sklearn.ensemble import AdaBoostClassifier

In [35]:
ada = AdaBoostClassifier(random_state = 96)

In [36]:
ada.fit(x_train, y_train)

ValueError: y should be a 1d array, got an array of shape (372778, 3) instead.